In [1]:



################### user inputs ###############################
repo_folder = './'


## filenames
master_input_filename = 'main_inputs'




###############################################################
import os
os.chdir(repo_folder)
import importlib


from pondit.load_data import *
from pondit.load_climate_data import *
from pondit.calc_ET_proj import *
from pondit.calc_model_proj import *
from pondit.plot_hydroperiod import *

import pandas
import numpy as np
# import matplotlib.pylab as plt
# import scipy.constants
# import scipy.optimize
# import datetime
# import scipy.ndimage.interpolation
# import statsmodels.api as sm
# import calendar

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


%matplotlib inline

folder_in = repo_folder + '/inputs/'
folder_out = repo_folder + '/results/'

models_to_use = np.array(pandas.read_csv(folder_in + 'climate_models.txt', header=None, index_col=0).index)


### load input data common to all sites
scalars = pandas.read_csv(folder_in + master_input_filename + '.csv', index_col=0)

site_list = scalars.index

site_list = scalars.loc[scalars['status'] != 'done', :].index


for site in site_list:
    scalars_hist = pandas.read_csv(folder_out + 'data/' + site + '_model_params_hist.csv', index_col=0)
    print(site)
    ##load data hist_model
    stage_storage, soils, data_hist, last_hist_date, calib_data = load_data(scalars_hist, site, folder_in)
    proj_pre_raw, proj_temp_raw = load_all_climate_data(scalars, site, folder_in)
    
    sws_calc_dict = {}
    sws_calc_elev = pandas.DataFrame(index=proj_pre_raw['date'])
    for m in models_to_use:
        print(m)
        
        data = load_climate_data(data_hist, m, scalars_hist, site, folder_in, proj_pre_raw, proj_temp_raw, last_hist_date)
        ## calculate ET, using Blaney Criddle
        bc_calc = calc_ET_proj(data, scalars, site, repo_folder, last_hist_date)

        # calculate model
        sws_calc, out = calc_pondit_proj(bc_calc, scalars_hist, site, stage_storage, soils, repo_folder)
        sws_calc.to_csv(folder_out + '/data/'+ site + '_' + m + '_proj_data.csv')
        sws_calc_dict[m] = sws_calc
        sws_calc_date_index = sws_calc.copy()
        sws_calc_date_index.index = sws_calc_date_index['date']
        sws_calc_elev.loc[:, m] = sws_calc_date_index['pond_elev']
    
    plot_hydroperiod(sws_calc_elev, scalars, site, folder_out)


Pond3
access1-0.1.rcp85
access1-3.1.rcp85
bcc-csm1-1.1.rcp85
bcc-csm1-1-m.1.rcp85
canesm2.1.rcp85
canesm2.2.rcp85
canesm2.3.rcp85
canesm2.4.rcp85
canesm2.5.rcp85
ccsm4.1.rcp85
ccsm4.2.rcp85
